<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/colab_cucat_bench_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cu_cat demo-datasets benchmark

In [1]:
!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu12==23.12.00 cudf-cu12==23.12.00 #cugraph-cu11 pylibraft_cu11 raft_dask_cu11 dask_cudf_cu11 pylibcugraph_cu11 pylibraft_cu11
# # !pip install git+https://github.com/graphistry/pygraphistry.git@feat/gpu-featurization
!pip install git+https://github.com/graphistry/pygraphistry.git@dev/depman_gpufeat
# # !pip install --upgrade git+https://github.com/graphistry/cu-cat.git@master
# # !git clone https://github.com/graphistry/cu-cat.git

!pip3 install --upgrade cu_cat

In [2]:
import cuml
cuml.__version__

'23.12.00'

In [3]:
!echo "0.01.0" > VERSION.txt
!cp VERSION.txt /usr/local/lib/python3.10/dist-packages/cu_cat/

In [4]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 200)
import cudf,cuml,cupy

In [5]:
  # from importlib import reload
# reload(cu_cat)
# os.chdir('cu-cat')
import cu_cat
# os.chdir('..')
print(cu_cat.__version__)

0.01.0


In [6]:
import graphistry
graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='fXjJnkE3Gik6BWy') ## key id, secret key
graphistry.__version__

'0.31.1+304.g8241a1a'

In [7]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [8]:
import logging
logger = logging.getLogger()
logging.basicConfig(level=logging.ERROR, force=True, format='%(asctime)s %(levelname)s %(message)s', datefmt='%H:%M:%S')

# logging.basicConfig(filename='app.log',
                    # level=logging.DEBUG,
                    # force=True, # Resets any previous configuration
                    # )

In [9]:
import cupy as cp
cp._default_memory_pool.free_all_blocks()


## ndf_reddit

In [10]:
import pandas as pd
import graphistry
from graphistry.features import topic_model

ndf_reddit = pd.read_csv('https://raw.githubusercontent.com/graphistry/pygraphistry/master/graphistry/tests/data/reddit.csv')#'pygraphistry/graphistry/tests/data/reddit.csv')
g = graphistry.nodes(ndf_reddit)
g2 = g.featurize(**topic_model, feature_engine="cu_cat")

In [ ]:
# g.umap(feature_engine="cu_cat")
g11a = g.umap(engine='cuml',memoize=False,feature_engine='cu_cat', cardinality_threshold=10,cardinality_threshold_target=10)


In [11]:
g11a.plot()

## redteam

In [12]:
df = pd.read_csv('https://gist.githubusercontent.com/silkspace/c7b50d0c03dc59f63c48d68d696958ff/raw/31d918267f86f8252d42d2e9597ba6fc03fcdac2/redteam_50k.csv', index_col=0)
red_team = pd.read_csv('https://gist.githubusercontent.com/silkspace/5cf5a94b9ac4b4ffe38904f20d93edb1/raw/888dabd86f88ea747cf9ff5f6c44725e21536465/redteam_labels.csv', index_col=0)
df['feats'] = df.src_computer + ' ' + df.dst_computer + ' ' + df.auth_type + ' ' + df.logontype
df['feats2'] = df.src_computer + ' ' + df.dst_computer
ndf = df.drop_duplicates(subset=['feats'])
tdf = pd.concat([red_team.reset_index(), ndf.reset_index()])
tdf['node'] = range(len(tdf))

In [13]:
# from cu_cat import GapEncoder, TableVectorizer
# tv = TableVectorizer()
# tv.fit_transform(tdf)

In [14]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()
g = graphistry.nodes(tdf, 'node')
g3a = g.umap(X=['feats'],
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='cu_cat',
            memoize=False,
            engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_cucat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         1621869 function calls (1598405 primitive calls) in 59.017 seconds

   Ordered by: internal time
   List reduced from 2562 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       13   27.602    2.123   31.721    2.440 /usr/local/lib/python3.10/dist-packages/cu_cat/_gap_encoder.py:1283(_multiplicative_update_h_smallfast)
        3   17.810    5.937   17.810    5.937 /usr/local/lib/python3.10/dist-packages/numpy/lib/arraysetops.py:523(in1d)
     5928    3.140    0.001    3.230    0.001 /usr/local/lib/python3.10/dist-packages/rmm/allocators/cupy.py:23(rmm_cupy_allocator)
        8    2.629    0.329    3.011    0.376 /usr/local/lib/python3.10/dist-packages/cu_cat/_gap_encoder.py:275(_get_H)
       10    1.638    0.164    1.638    0.164 {built-in method gc.collect}
       10    0.969    0.097    2.799    0.280 /usr/local/lib/python3.10/dist-packages/cu_cat/_gap_encoder.py:1142(_multiplicative_update_w_smallfast)
      263    0.5

In [ ]:
g3a.plot()

In [16]:
# %%time
# pr = cProfile.Profile()
# pr.enable()
# g = graphistry.nodes(tdf, 'node')
# g3b = g.umap(X=['feats'],
#             min_words=1000000, # force high so that we don't use Sentence Transformers
#             cardinality_threshold=4, # set low so we force topic model
#             n_topics=32, # number of topics
#             use_scaler=None,
#             feature_engine='dirty_cat',
#             memoize=False,
#             engine='cuml',
#             use_scaler_target=None
#            )

# pr.disable()
# stats = Stats(pr)
# stats.sort_stats('tottime').print_stats(20)

# with open('redteam_umap_dirtycat.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(20)

In [17]:
# g3b.plot()

## 20newsgroups

In [18]:
from sklearn.datasets import fetch_20newsgroups
# newsgroups = fetch_20newsgroups()#categories=categories)
n_samples = 2000
# n_features = 1000
# n_components = 10
# n_top_words = 20
# batch_size = 128
# init = "nndsvda"

news, _ = fetch_20newsgroups(
    shuffle=True,
    random_state=1,
    remove=("headers", "footers", "quotes"),
    return_X_y=True,
)

news = news[:n_samples]
news=pd.DataFrame(news)

In [19]:
g = graphistry.nodes(news)#.replace('nan',np.nan).fillna('0o0o0'))
pr = cProfile.Profile()
pr.enable()


# g2 = g.featurize(feature_engine='cu_cat',memoize=False)
g4a = g.umap(engine='cuml',feature_engine='cu_cat',memoize=False,cardinality_threshold_target=100)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_cucat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         998624 function calls (975874 primitive calls) in 73.881 seconds

   Ordered by: internal time
   List reduced from 2582 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       13   43.183    3.322   62.618    4.817 /usr/local/lib/python3.10/dist-packages/cu_cat/_gap_encoder.py:1283(_multiplicative_update_h_smallfast)
     5983   16.705    0.003   16.791    0.003 /usr/local/lib/python3.10/dist-packages/rmm/allocators/cupy.py:23(rmm_cupy_allocator)
      263    2.376    0.009    2.944    0.011 /usr/local/lib/python3.10/dist-packages/cupyx/scipy/sparse/_csr.py:148(__mul__)
        8    1.748    0.219    1.748    0.219 {built-in method cupy_backends.cuda.libs.nvrtc.compileProgram}
       10    1.508    0.151    1.508    0.151 {built-in method gc.collect}
1160/1142    1.395    0.001    1.606    0.001 /usr/lib/python3.10/contextlib.py:76(inner)
       10    1.210    0.121    4.161    0.416 /usr/local/lib/python3.10/dist-packages

In [ ]:
g4a.plot()

In [ ]:
# # !pip install dirty_cat
# g = graphistry.nodes(news)
# pr = cProfile.Profile()
# pr.enable()


# # g2 = g.featurize(feature_engine='dirty_cat',memoize=False)
# g4b = g.umap(engine='cuml',feature_engine='dirty_cat',memoize=False, cardinality_threshold_target=100)

# pr.disable()
# stats = Stats(pr)
# stats.sort_stats('tottime').print_stats(20)

# with open('20news_umap_dirtycat.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(20)

In [ ]:
# g4b.plot()

## ask HN

In [22]:
# # get the data top 3000 posts on Hacker News
askHN = pd.read_csv('https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_df.csv', index_col=0)

askHN = askHN.sample(1000,replace=False) # set smaller if you want to test a minibatch
# df = df[['text','title']]
# df['title'] = df['title'].sample(frac=1).values

In [23]:
g = graphistry.nodes((askHN[['text','title']]))# 'id','parent']]))#, \
        # 'id', 'parent']]))#, 'descendants', 'ranking', 'deleted']]))

pr = cProfile.Profile()
pr.enable()


# g1a = g.featurize(feature_engine='cu_cat',memoize=False)
g5a = g.umap(engine='cuml',memoize=False,feature_engine='cu_cat', cardinality_threshold=10,cardinality_threshold_target=10)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_cucat_3k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         1682195 function calls (1639163 primitive calls) in 21.582 seconds

   Ordered by: internal time
   List reduced from 2468 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       26   10.024    0.386   13.023    0.501 /usr/local/lib/python3.10/dist-packages/cu_cat/_gap_encoder.py:1283(_multiplicative_update_h_smallfast)
       20    3.244    0.162    3.244    0.162 {built-in method gc.collect}
    11896    1.679    0.000    1.835    0.000 /usr/local/lib/python3.10/dist-packages/rmm/allocators/cupy.py:23(rmm_cupy_allocator)
2398/2362    0.823    0.000    1.232    0.001 /usr/lib/python3.10/contextlib.py:76(inner)
       16    0.600    0.037    0.680    0.043 /usr/local/lib/python3.10/dist-packages/cu_cat/_gap_encoder.py:275(_get_H)
      526    0.547    0.001    0.983    0.002 /usr/local/lib/python3.10/dist-packages/cupyx/scipy/sparse/_csr.py:148(__mul__)
        6    0.435    0.073    0.435    0.073 /usr/local/lib/python3.10

In [ ]:
g5a.plot()

In [25]:
# # !pip install dirty_cat
# g = graphistry.nodes(askHN[['text','title']])
# pr = cProfile.Profile()
# pr.enable()


# # g5b = g.featurize(feature_engine='dirty_cat',memoize=False)
# g5b = g.umap(engine='cuml',memoize=False,feature_engine='dirty_cat', cardinality_threshold=10,cardinality_threshold_target=10)

# pr.disable()
# stats = Stats(pr)
# stats.sort_stats('tottime').print_stats(20)

# with open('aHN_umap_dirtycat_3k.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(20)

In [26]:
# g5b.plot()

## winlogs

In [ ]:
!wget -nc https://www.dropbox.com/s/31dx1g6g59exoc3/part.88.parquet
winlogs=pd.read_parquet('part.88.parquet')
winlogsA=winlogs.sample(10000,replace=False)#.iloc[:,:10]
# winlogsA=winlogsA#[['LogonID','UserName','LogHost']]#,'DomainName','LogonType','SubjectLogonID','Status','Destination','ServiceName']]#,'EventID']]#.convert_dtypes() #.replace('nan',np.nan).fillna('0o0o0')


In [28]:
# import cupy as cp
# del g,g2,g3
# cp._default_memory_pool.free_all_blocks()

In [29]:
g = graphistry.nodes((winlogsA)) #.replace('nan',np.nan).fillna('0o0o0'))
pr = cProfile.Profile()
pr.enable()

# g1a = g.featurize(feature_engine='cu_cat',memoize=False)
g1a = g.umap(engine='cuml',feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize_cucat_20k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         7892849 function calls (7726083 primitive calls) in 102.804 seconds

   Ordered by: internal time
   List reduced from 2747 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       91   46.473    0.511   54.827    0.602 /usr/local/lib/python3.10/dist-packages/cu_cat/_gap_encoder.py:1283(_multiplicative_update_h_smallfast)
       70   11.930    0.170   11.930    0.170 {built-in method gc.collect}
       21    8.741    0.416    8.741    0.416 /usr/local/lib/python3.10/dist-packages/numpy/lib/arraysetops.py:523(in1d)
       28    6.234    0.223    6.234    0.223 {built-in method cupy_backends.cuda.libs.nvrtc.compileProgram}
    41494    5.990    0.000    6.472    0.000 /usr/local/lib/python3.10/dist-packages/rmm/allocators/cupy.py:23(rmm_cupy_allocator)
       56    4.624    0.083    5.322    0.095 /usr/local/lib/python3.10/dist-packages/cu_cat/_gap_encoder.py:275(_get_H)
12158/12041    2.363    0.000    4.263    0.000 /usr/lib

In [30]:
g1a.plot()

In [31]:
# g = graphistry.nodes(winlogs)
# pr = cProfile.Profile()
# pr.enable()

# # g1b = g.featurize(feature_engine='dirty_cat',memoize=False)
# g1b = g.umap(engine='cuml',feature_engine='dirty_cat',memoize=False)

# pr.disable()
# stats = Stats(pr)
# stats.sort_stats('tottime').print_stats(20)

# with open('winlogs_featurize_dirtycat_20k.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(20)

In [32]:
# g1b.plot()

## CTU-13 malware dataset

In [35]:
edf = pd.read_csv('https://gist.githubusercontent.com/silkspace/33bde3e69ae24fee1298a66d1e00b467/raw/dc66bd6f1687270be7098f94b3929d6a055b4438/malware_bots.csv', index_col=0)
T = edf.Label.apply(lambda x: True if 'Botnet' in x else False)
bot = edf[T]
nbot = edf[~T]
print(f'Botnet abundance: {100*len(bot)/len(edf):0.2f}%')# so botnet traffic makes up a tiny fraction of total

# let's balance the dataset in a 10-1 ratio, for speed and demonstrative purposes
negs = nbot.sample(10*len(bot))
edf = pd.concat([bot, negs])  # top part of arrays are bot traffic, then all non-bot traffic
edf = edf.drop_duplicates()

# some useful indicators for later that predict Botnet as Bool and Int
Y = edf.Label.apply(lambda x: 1 if 'Botnet' in x else 0)  # np.array(T)

# Later we will use and exploit any meaning shared between the labels in a latent distribution

# add it to the dataframe
edf['bot'] = Y

# name some columns for edges and features
src = 'SrcAddr'
dst = 'DstAddr'
good_cols_with_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes', src, dst]

good_cols_without_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes']

## some encoding parameters
n_topics = 20
n_topics_target = 7



Botnet abundance: 0.69%


In [37]:
import cupy as cp
cp._default_memory_pool.free_all_blocks()


In [ ]:
# lets umap the data
g = graphistry.nodes(edf[['SrcAddr','DstAddr']])#.apply(lambda x: str(x).zfill(4)))#, src, dst)
pr = cProfile.Profile()
pr.enable()

g2a = g.umap(engine='cuml',memoize=False,feature_engine='cu_cat')

# g2a = g.umap(kind='edges',
#             # X=good_cols_with_edges,
#             # y = ['bot'],
#             use_scaler='quantile',
#             use_scaler_target=None,
#             cardinality_threshold=20,
#             cardinality_threshold_target=2,
#             n_topics=n_topics,
#             feature_engine='cu_cat',
#             engine='cuml',
#             memoize=False,
#             n_topics_target=n_topics_target,
#             n_bins=n_topics_target,
#             metric='euclidean',
#             n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_cucat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


In [ ]:
g2a.plot()

In [ ]:
# # lets umap the data
# g = graphistry.nodes(edf[['SrcAddr','DstAddr']])#.apply(lambda x: str(x).zfill(4)))#, src, dst)
# pr = cProfile.Profile()
# pr.enable()

# # !pip install dirty_cat
# g2b = g.umap(engine='cuml',memoize=False,feature_engine='dirty_cat')

# # g2b = g.umap(kind='edges',
# #             X=good_cols_with_edges,
# #             y = ['bot'],
# #             use_scaler='quantile',
# #             use_scaler_target=None,
# #             cardinality_threshold=20,
# #             cardinality_threshold_target=2,
# #             n_topics=n_topics,
# #             feature_engine='dirty_cat',
# #             engine='cuml',
# #             memoize=False,
# #             n_topics_target=n_topics_target,
# #             n_bins=n_topics_target,
# #             metric='euclidean',
# #             n_neighbors=12)
# pr.disable()
# stats = Stats(pr)
# stats.sort_stats('tottime').print_stats(20)

# with open('ctu13_umap_dirtycat.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(20)


In [ ]:
# g2b.plot()